## 환경설정 확인(GPU, RAM)

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.180
cache size	: 39424 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [ ]:
!nvidia-smi

Mon Nov  1 15:09:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 6. 모델링

### 문장 요약 모델 (koBART 모델)
> 참고자료 링크 : https://github.com/ChoiSaeyoun/kobart_summarization  
> 사용 데이터셋 링크 : https://aihub.or.kr/aidata/30714  
> streamlit 참고자료 : https://developer-ping9.tistory.com/115

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/munning_rachine/KoBART'

/content/drive/MyDrive/munning_rachine/KoBART


In [ ]:
# !git clone https://github.com/seujung/KoBART-summarization.git
# mv KoBART-summarization KoBART

Cloning into 'KoBART-summarization'...
remote: Enumerating objects: 104, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 104 (delta 52), reused 66 (delta 23), pack-reused 0
Receiving objects: 100% (104/104), 37.22 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [ ]:
pwd

'/content/drive/MyDrive/munning_rachine/KoBART'

#### 라이브러리 설치

In [1]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart
!pip install -r requirements.txt
!pip install gdown
!pip install rouge
# !pip install pyngrok #밑에 동일한 코드 있음
!pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-ygow91wb/kobart_1051ca88b9254de58a60d072b79e0877
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-install-ygow91wb/kobart_1051ca88b9254de58a60d072b79e0877
  Using cached transformers-4.3.3-py3-none-any.whl (1.9 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.2
    Uninstalling transformers-4.8.2:
      Successfully uninstalled transformers-4.8.2
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


### 비플라이 문서요약 데이터 전처리 하기

In [ ]:
import pandas as pd
import json

In [ ]:
#json_path ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/신문기사_train_original/' #뉴스
#json_path ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/사설잡지_train_original/' #시사
json_path ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/법률_train_original/' #법률

In [ ]:
with open(json_path+'train_original.json') as f:
    json_data = json.load(f)

In [ ]:
df = pd.DataFrame(json_data['documents'])

In [ ]:
df

,id,category,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,100004,일반행정,small,377,19841226,부당노동행위구제재심판정취소,"[[{'index': 0, 'sentence': '원고가 소속회사의 노동조합에서 분...",3783,"{'readable': 3, 'accurate': 3, 'informative': ...","[5, 4, 2]",[원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에...
1,100005,세무,small,510,19841226,법인세부과처분취소,"[[{'index': 0, 'sentence': '수출입업체인 원고가 의류제품을 제...",3697,"{'readable': 3, 'accurate': 3, 'informative': ...","[0, 5, 6]",[수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하...
2,100006,세무,small,248,19841226,양도소득세부과처분취소,"[[{'index': 0, 'sentence': '가등기담보권자가 제소전 화해조항에...",3680,"{'readable': 5, 'accurate': 5, 'informative': ...","[0, 2, 3]",[가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고...
3,100008,세무,small,369,19841226,부가가치세부과처분취소,"[[{'index': 0, 'sentence': '가. 부가가치세법 제22조 제3항...",3680,"{'readable': 5, 'accurate': 5, 'informative': ...","[1, 2, 3]",[부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제...
4,100014,세무,small,202,19841226,양도소득세부과처분취소,"[[{'index': 0, 'sentence': '소득세법 제116조 제1항의 규정...",3680,"{'readable': 5, 'accurate': 5, 'informative': ...","[1, 2, 3]",[소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 ...
...,...,...,...,...,...,...,...,...,...,...,...
27028,83579,민사,small,519,20041126,손해배상(기),"[[{'index': 0, 'sentence': '[1] 공해로 인한 손해배상청구소...",3851,"{'readable': 4, 'accurate': 4, 'informative': ...","[1, 3, 6]",[공해로 인한 손해배상청구소송에 있어 가해기업은 피해자보다 원인조사에 유리할 뿐 아...
27029,83581,민사,medium,1012,20041126,추심금,"[[{'index': 0, 'sentence': '[1] 구 민사소송법(2002. ...",3814,"{'readable': 4, 'accurate': 4, 'informative': ...","[1, 11, 12]",[구 민사소송법에서 보충송달은 송달장소에서 수령대행권이 있는 자에게만 할 수 있다고...
27030,83583,민사,medium,786,20041126,구상금,"[[{'index': 0, 'sentence': '[1] 구 신기술사업금융지원에관한...",3708,"{'readable': 3, 'accurate': 3, 'informative': ...","[0, 4, 5]",[구 신기술사업금융법에서는 보증채무를 이행하지 못한 기업으로부터 기술보증기금이 추가...
27031,83585,민사,small,354,20041126,가압류이의,"[[{'index': 0, 'sentence': '구 주택건설촉진법(2003. 5....",4255,"{'readable': 5, 'accurate': 5, 'informative': ...","[0, 2, 3]",[구 주택건설촉진법 취지는 입주자모집공고승인 이후 공급받는 이들을 보호하기 위함으로...


In [ ]:
new_df = df[['id', 'category', 'publish_date', 'text', 'abstractive', 'extractive']]
new_df

,id,category,publish_date,text,abstractive,extractive
0,100004,일반행정,19841226,"[[{'index': 0, 'sentence': '원고가 소속회사의 노동조합에서 분...",[원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에...,"[5, 4, 2]"
1,100005,세무,19841226,"[[{'index': 0, 'sentence': '수출입업체인 원고가 의류제품을 제...",[수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하...,"[0, 5, 6]"
2,100006,세무,19841226,"[[{'index': 0, 'sentence': '가등기담보권자가 제소전 화해조항에...",[가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고...,"[0, 2, 3]"
3,100008,세무,19841226,"[[{'index': 0, 'sentence': '가. 부가가치세법 제22조 제3항...",[부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제...,"[1, 2, 3]"
4,100014,세무,19841226,"[[{'index': 0, 'sentence': '소득세법 제116조 제1항의 규정...",[소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 ...,"[1, 2, 3]"
...,...,...,...,...,...,...
27028,83579,민사,20041126,"[[{'index': 0, 'sentence': '[1] 공해로 인한 손해배상청구소...",[공해로 인한 손해배상청구소송에 있어 가해기업은 피해자보다 원인조사에 유리할 뿐 아...,"[1, 3, 6]"
27029,83581,민사,20041126,"[[{'index': 0, 'sentence': '[1] 구 민사소송법(2002. ...",[구 민사소송법에서 보충송달은 송달장소에서 수령대행권이 있는 자에게만 할 수 있다고...,"[1, 11, 12]"
27030,83583,민사,20041126,"[[{'index': 0, 'sentence': '[1] 구 신기술사업금융지원에관한...",[구 신기술사업금융법에서는 보증채무를 이행하지 못한 기업으로부터 기술보증기금이 추가...,"[0, 4, 5]"
27031,83585,민사,20041126,"[[{'index': 0, 'sentence': '구 주택건설촉진법(2003. 5....",[구 주택건설촉진법 취지는 입주자모집공고승인 이후 공급받는 이들을 보호하기 위함으로...,"[0, 2, 3]"


In [ ]:
for i in range(len(new_df)):
  context = pd.DataFrame(new_df.loc[i, 'text'])
  doc = ''
  for j in range(len(context)):
    try:
      doc += context[0][j]['sentence']
    except:
      pass
  new_df.loc[i, 'text'] = doc
  new_df.loc[i, 'abstractive'] = new_df.loc[i, 'abstractive'][0]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path whi

In [ ]:
new_df

,id,category,publish_date,text,abstractive,extractive
0,100004,일반행정,19841226,원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...,원고가 주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...,"[5, 4, 2]"
1,100005,세무,19841226,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...,수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...,"[0, 5, 6]"
2,100006,세무,19841226,가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...,가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...,"[0, 2, 3]"
3,100008,세무,19841226,가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...,부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...,"[1, 2, 3]"
4,100014,세무,19841226,소득세법 제116조 제1항의 규정에 의하면,소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...,"[1, 2, 3]"
...,...,...,...,...,...,...
27028,83579,민사,20041126,[1] 공해로 인한 손해배상청구소송에 있어서는 가해행위와 손해발생 사이의 인과관계의...,공해로 인한 손해배상청구소송에 있어 가해기업은 피해자보다 원인조사에 유리할 뿐 아니...,"[1, 3, 6]"
27029,83581,민사,20041126,[1] 구 민사소송법(2002. 1. 26. 법률 제6626호로 전문 개정되기 전의...,구 민사소송법에서 보충송달은 송달장소에서 수령대행권이 있는 자에게만 할 수 있다고 ...,"[1, 11, 12]"
27030,83583,민사,20041126,[1] 구 신기술사업금융지원에관한법률(2002. 8. 26. 법률 제6705호 기술...,구 신기술사업금융법에서는 보증채무를 이행하지 못한 기업으로부터 기술보증기금이 추가보...,"[0, 4, 5]"
27031,83585,민사,20041126,구 주택건설촉진법(2003. 5. 29. 법률 제6916호 주택법으로 전문 개정되기...,구 주택건설촉진법 취지는 입주자모집공고승인 이후 공급받는 이들을 보호하기 위함으로 ...,"[0, 2, 3]"


In [ ]:
new_df['text'][0]

'원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고,'

In [ ]:
new_df.to_csv(json_path+'train.csv', encoding='utf-8-sig')

> 특수문자 제거하기

참고: https://sikaleo.tistory.com/64

In [ ]:
csv_path ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/신문기사_train_original/' #뉴스
csv_path2 ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/사설잡지_train_original/' #시사
# csv_path ='/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/법률_train_original/' #법률

In [ ]:
new_df = pd.read_csv(csv_path+'train.csv', encoding='utf-8')
new_df2 = pd.read_csv(csv_path2+'train.csv', encoding='utf-8')

In [ ]:
new_df = new_df.dropna()
new_df2 = new_df2.dropna()

In [ ]:
df = pd.concat([new_df, new_df2])

In [ ]:
import re
def cleasing(text):
  repl = ''
  pattern = '[\[\]▲△ㆍ~@]' # 특수기호만 제거
  text = re.sub(pattern= pattern, repl=repl, string=text)
  return text

In [ ]:
new_df['text'] = new_df['text'].map(lambda x: cleasing(x))
new_df2['text'] = new_df2['text'].map(lambda x: cleasing(x))

In [ ]:
new_df = new_df.loc[new_df['category'] == '종합']
new_df2 = new_df2.loc[new_df2['category'] == '종합']

In [ ]:
df = new_df.sample(n=7000) #뉴스
df = pd.concat([df,new_df2.sample(n=3000)]) #사설잡지

In [ ]:
df

,Unnamed: 0,id,category,publish_date,text,abstractive,extractive
0,0,290741778,종합,2018-01-05 18:54:55,ha당 조사료 400만원…작물별 차등 지원이성훈 sinawihanmail.net전라...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,"[2, 3, 10]"
1,1,290741792,종합,2018-01-05 18:53:20,"8억 투입, 고소천사벽화·자산마을에 색채 입혀이성훈 sinawihanmail.net...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"[2, 4, 11]"
4,4,290741797,종합,2018-01-05 18:52:36,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항이성훈 sinawihanmail...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"[3, 7, 4]"
7,7,290741802,종합,2018-01-05 18:50:02,"교육지원청 Wee센터, 겨울방학 카운슬링 운영김영신 기자 yskim0966naver...","광양교육지원청 Wee센터는 4일부터 초,중,고등학생과 학부모를 대상으로 건강한 가족...","[2, 3, 8]"
8,8,290741808,종합,2018-01-05 18:49:48,"서양화가 강홍순 전, 정인수 화가‘通통通통한 산수화’전김영신 기자 yskim0966...",말과 흑인여성을 화폭에 담은 서양화가 ‘강홍순 전’이 오는 14일까지 광양문화원 기...,"[4, 8, 13]"
...,...,...,...,...,...,...,...
63059,63059,365236794,종합,2019-10-31 00:33:00,갑작스러운 정시 확대로 고교는 물론 학부모와 교육계 전체가 큰 혼란에 빠졌다.교육계...,교육부가 2022학년도 대입부터 정시 비율을 30%까지 늘리는 대입개편안을 발표한 ...,"[1, 4, 16]"
63060,63060,365238424,종합,2019-10-31 01:10:22,"1 년 전인 2018 년 10 월30 일, 대법원은 미쓰비시와 일본제철에 대해 배상...",인천시의회는 23일 조금이나마 위로하는 마음을 담아 '독립유공자 지원 조례안'을 개...,"[11, 13, 18]"
63061,63061,365238425,종합,2019-10-31 01:10:22,어제(30 일)는 제39 회 항공의 날이었다.29 일 국제 세미나에서는 매우 고무적...,"제39회 항공의 날을 맞아 인천의 도시경쟁력 신장, 지역 청년 일자리 창출의 뼈대가...","[0, 15, 17]"
63065,63065,365247691,종합,2019-10-31 04:40:00,강신욱 통계청장이 29 일 서울 종로구 정부서울청사 브리핑룸에서 2019 년 8 월...,올해 경제활동인구조사의 근로형태별 부가조사에서 집계된 비정규직 노동자 수는 전년대비...,"[1, 11, 13]"


In [ ]:
df = df.reset_index()

In [ ]:
csv_path = '/content/drive/MyDrive/munning_rachine/dataset/blysoft/Training/'
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1, random_state=20)
train.to_csv(path_or_buf=csv_path+'merge_train.csv', encoding = 'utf-8-sig', index=False)
test.to_csv(path_or_buf=csv_path+'merge_test.csv', encoding = 'utf-8-sig', index=False)

### 비플라이 모델링

1만개 csv파일 : small_train.csv

2만개 csv파일 : train.csv

22만개(뉴스+사설잡지 종합 카테고리만) csv파일 : merge_train.csv

In [2]:
cd '/content/drive/MyDrive/munning_rachine'

/content/drive/MyDrive/munning_rachine


In [ ]:
import time
import datetime
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
start = time.time()
!python KoBART/train.py \
--train_file='dataset/blysoft/Training/merge_train.csv' \
--test_file='dataset/blysoft/Training/merge_test.csv' \
--mode='train' \
--batch_size=10 \
--num_workers=2 \
--gradient_clip_val=1.0 \
--gpus=1 \
--accelerator=ddp \
--max_epochs=50 \
--max_len=512 \
--default_root_dir='KoBART/model/blysoft/train'

sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)

INFO:root:Namespace(accelerator='ddp', accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=10, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='KoBART/model/blysoft/train', deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, hparams_file=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=50, max_len=512, max_steps=None, min_epochs=None, min_steps=None, mode='train', model_path=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=2, overfit_batches=0.0, plugins=None, precision=32, prepa

In [ ]:
#train model에 추가 학습 필요할 시 사용할 코드
!python KoBART/train.py \
--train_file='dataset/blysoft/Training/merge_train.csv' \
--test_file='dataset/blysoft/Training/merge_test.csv' \
--mode='train' \
--checkpoint_path='KoBART/model/blysoft/train/kobart_summary-model_chp/epoch=02-val_loss=1.756.ckpt' \
--hparams_file='KoBART/model/blysoft/train/tb_logs/default/version_1/hparams.yaml' \
--batch_size=10 \
--num_workers=2 \
--gradient_clip_val=1.0 \
--gpus=1 \
--accelerator=ddp \
--max_epochs=5 \
--default_root_dir='KoBART/model/train'

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint...
INFO:lightning:Saving latest checkpoint...
Epoch 4, global step 80086: val_loss reached 1.81462 (best 1.74364), saving model to "/content/drive/My Drive/munning_rachine/KoBART/model/train/kobart_summary-model_chp/epoch=04-val_loss=1.815.ckpt" as top 5
INFO:lightning:Epoch 4, global step 80086: val_loss reached 1.81462 (best 1.74364), saving model to "/content/drive/My Drive/munning_rachine/KoBART/model/train/kobart_summary-model_chp/epoch=04-val_loss=1.815.ckpt" as top 5
^C


In [ ]:
# 체크포인트 파일 새로운 파일이 나오면 해당 파일로 변경해야함
# 하이퍼 파라미터 파일도 새로운 버전(version_10)이 나오면 변경해야함
# 원래 경로
# --train_file='dataset/new_train.csv' \
# --test_file='dataset/new_test.csv' \

!python KoBART/train.py \
--train_file='dataset/blysoft/Training/merge_train.csv' \
--test_file='dataset/blysoft/Training/merge_test.csv' \
--mode='test' \
--checkpoint_path='KoBART/model/blysoft_220000/train/kobart_summary-model_chp/epoch=02-val_loss=1.788.ckpt' \
--hparams_file='KoBART/model/blysoft_220000/train/tb_logs/default/version_0/hparams.yaml' \
--batch_size=10 \
--num_workers=8 \
--gpus=1 \
--accelerator=ddp \
--default_root_dir='KoBART/model/blysoft_220000/test'

INFO:root:Namespace(accelerator='ddp', accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=10, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path='KoBART/model/blysoft_220000/train/kobart_summary-model_chp/epoch=02-val_loss=1.788.ckpt', default_root_dir='KoBART/model/blysoft_220000/test', deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=0, hparams_file='KoBART/model/blysoft_220000/train/tb_logs/default/version_0/hparams.yaml', limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=None, max_len=512, max_steps=None, min_epochs=None, min_steps=None, mode='test', model_path=None, move_metrics_to_cpu=False, multipl

In [ ]:
pwd

'/content/drive/My Drive/munning_rachine'

In [ ]:
!python KoBART/download_binary.py

Download config.json
Downloading...
From: https://drive.google.com/uc?id=1H13loH6dS_2c2Z21kaBtgz42QsjkdAwO
To: /content/drive/My Drive/munning_rachine/KoBART/model/blysoft_220000/output/config.json
100% 1.20k/1.20k [00:00<00:00, 2.36MB/s]
Download pytorch_model.bin
Downloading...
From: https://drive.google.com/uc?id=1D7BAXK_0faWW39c0ptE3FtROsVRbTNwI
To: /content/drive/MyDrive/munning_rachine/KoBART/model/blysoft_220000/output/pytorch_model.bin
100% 496M/496M [00:01<00:00, 260MB/s]


In [8]:
!python KoBART/get_model_binary.py \
--hparams='KoBART/model/blysoft_220000/train/tb_logs/default/version_0/hparams.yaml' \
--model_binary='KoBART/model/blysoft_220000/train/kobart_summary-model_chp/epoch=02-val_loss=1.788.ckpt' \
--output_dir='KoBART/model/blysoft_220000/output'

KoBART/get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)
[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


### streamlit

In [9]:
!pip install streamlit --upgrade

     |████████████████████████████████| 8.3 MB 8.5 MB/s 
  Attempting uninstall: streamlit
    Found existing installation: streamlit 0.72.0
    Uninstalling streamlit-0.72.0:
      Successfully uninstalled streamlit-0.72.0


In [10]:
!pip install pyngrok
!ngrok authtoken 1xiXwKjnUWUlFP1thbl5KYgsbhD_7cUjM8p7tymQrbNH4gHNG

     |████████████████████████████████| 745 kB 14.9 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19006 sha256=d8073ae57d495d46c2832918add57b411204b650698e6ecce1d06d06993050c1
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [11]:
from pyngrok import ngrok

In [12]:
!streamlit run KoBART/infer.py&>/dev/null&
url = ngrok.connect( addr='8501')
url

<NgrokTunnel: "http://9289-34-91-22-161.ngrok.io" -> "http://localhost:8501">

In [ ]:
ngrok.kill()